# Mathematical Model

To better represent and make tools to solve the Capacited Vehicle Routing Problem using Linear Programming methods, we will use a mathematical model. This mathematical model turns the CVRP into linear relationships that represent its objective and constraints. 

## Data

### Graph

A graph ($G$) will be used to represent the depot and the cities to deliver to. These geographical points will be the graph's verticles (which will be referred to as "nodes"). Those nodes are all linked together (making the graph complete), by edges whose values represent the distance between one node and another.
- $K$ is the set of vehicles available. *$k$ will therefore, refer to an element of $K$.*
- $V$ is the set of verticles (nodes).
- $E$ is the set of edges.
- $Q$ is the vehicles' cargo capacity 
- $d_i$ is the demand (total volume of products to be delivered) associated to the node $i$.
- $c_{ij}$ is the cost associated to the edge $(i,j)$, which is the distance between the node $i$ and the node $j$. *In some implementations, $c_{ij}$ can be different than $c_{ji}$, as the route from city i to city y can be different than from city y to city i, depending on factors such as one-way roads*

### Decision variable

- $x_{ij}^k$ is equal to $1$ if the route represented by the edge $(i,j)$ is traveled by the vehicle $k$, if not, it's equal to $0$.

## Objective function

Our model's objective function is the following:
$$\min \sum_{k \in K} \sum_{i \in V} \sum_{j \in V} c_{ij} x_{ij}^k$$
As $\min$ specifices it, the objective here is to minimize the result of this function, which means minimizing the total distance driven by the trucks.

## Constraints

1. $\sum_{k \in K} \sum_{j \in V} x_{ij}^k = 1, \quad \forall i \in V \setminus \{0\}$

This first constraint makes it so that each node is strictly visited once by a vehicle.

   2.1. $\sum_{j \in V} x_{0j}^k = 1, \quad \forall k \in K$

Every vehicle must have the depot as the beginning of their route

   2.2. $\sum_{j \in V} x_{i0}^k = 1, \quad \forall k \in K$

Every vehicle must have the depot as the ending of their route

3. $\sum_{j \in V} x_{ij}^k = \sum_{j \in V} x_{ji}^k, \quad \forall i \in V, \forall k \in K$

This constraint makes sure that vehicles come out of the nodes they came in

4. $\sum_{i \in V} d_i \sum_{j \in V} x_{ij}^k \leq Q, \quad \forall k \in K$

The cargo capacity of the vehicles musn't be exceeded

6. $\sum_{k \in K} \sum_{(i,j) \in S,i \neq j} x_{ij}^k \leq |S|-1, \quad S \subseteq V \setminus \{0\}$

Constraint used to eliminate sub-tours

## Sources

- "Optimisation de tournées de véhicules pour l’exploitation de Réseau Telecom", by Arnaud Malapert, issued on the 6th of September 2006.
- "Capacitated Vehicle Routing Problem (CVRP) with Python+Pulp and Google Maps API", by Kijun Kim, issued on the 30th of April 2020, on the blog website Medium.

## Solving using a solver program

This model can be used as-is to solve small-sized instances of the CVRP problem by coding it into a program that uses a linear problem solver software.

### Python Code with PuLP

In [2]:
import pandas as pd
import pulp
import itertools
import math

## Program settings
# Parameter to decide if the distance between cities should be as the crow flies (False) or based on the roads (True, requires an OpenRouteService server)
road_distance = True
# Address to the OpenRouteService server (only relevant if road_distance is set to True)
ors_address = 'http://127.0.0.1:8082/ors'

## Declaration of constants
# Number of vehicle
vehicle_count = 4

# Capacity of vehicles
vehicle_capacity = 12

# Depot latitude and longitude
depot_latitude = 48.95656958120599
depot_longitude = 2.4753272179034886

# Cities' latitude, longitude and demand
cities = {
"latitude": [43.7,43.2967,43.6,44.8333,43.6,47.2167,50.6333,48.5833,45.7589,48.86],
"longitude": [7.25,5.37639,1.43333,-0.566667,3.88333,-1.55,3.06667,7.75,4.84139,2.34445],
"demand": [2,3,1,1,1,2,1,3,1,2]
}

# Adding of the depot to the list of cities
cities['latitude'] = [depot_latitude]+cities['latitude']
cities['longitude'] = [depot_longitude]+cities['longitude']
cities['demand'] = [0]+cities['demand']

# Customer count ('0' is depot)
customer_count = len(cities["latitude"])

# Construction of the data frame based on the "cities" list
df = pd.DataFrame(cities)

# The depot is set as the center
df.iloc[0, 0] = cities['latitude'][0]
df.iloc[0, 1] = cities['longitude'][0]
df.iloc[0, 2] = 0

# If the road_distance parameter is true, the functions used for that purpose are defined
if(road_distance) :
    import openrouteservice as ors
    # Address to the OpenRouteService server, used to get the exact distance between cities by using the roads
    client = ors.Client(base_url=ors_address)
    ## Function used to ask the distance between two points to the OpenRouteService server
    def road_distance(lat1, lon1, lat2, lon2):
        return client.directions(coordinates=[[lon1, lat1], [lon2, lat2]], profile='driving-car', format='geojson')['features'][0]['properties']['segments'][0]['distance']

## Haversine function, used to calculate the distance between two GPS coordinates, in meters
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371.0  # Radius of Earth in kilometers
    distance = c * r * 1000  # Convert to meters
    return distance

## Function used for obtaining a matrix of distances between all points
def distance_calculator(df):
    distance_result = [[0 for _ in range(len(df))] for _ in range(len(df))]
    for i in range(len(df)):
        for j in range(len(df)):
            if i != j:
                if(road_distance):
                    distance_result[i][j] = road_distance(df['latitude'].iloc[i], df['longitude'].iloc[i],
                                                       df['latitude'].iloc[j], df['longitude'].iloc[j])

                else :
                    distance_result[i][j] = haversine(df['latitude'].iloc[i], df['longitude'].iloc[i],
                                                       df['latitude'].iloc[j], df['longitude'].iloc[j])
    return distance_result

# Construction of a list that contains all the distances between cities
distances = distance_calculator(df)

## Problem solving with PuLP
for vehicle_count in range(1, vehicle_count + 1):
    # Definition of a Linear Programming problem instance
    problem = pulp.LpProblem("CVRP", pulp.LpMinimize)

    # Definition of binary variables (can either be 0 or 1)
    x = [[[pulp.LpVariable("x%s_%s,%s" % (i, j, k), cat="Binary") if i != j else None for k in range(vehicle_count)]
          for j in range(customer_count)] for i in range(customer_count)]

    # Adding of objective function to the problem
    problem += pulp.lpSum(distances[i][j] * x[i][j][k] if i != j else 0
                          for k in range(vehicle_count)
                          for j in range(customer_count)
                          for i in range(customer_count))

    # Adding of constraints to the problem
    # Constraint n°1
    for j in range(1, customer_count):
        problem += pulp.lpSum(x[i][j][k] if i != j else 0
                              for i in range(customer_count)
                              for k in range(vehicle_count)) == 1

    # Constraints n°2.1 & 2.2
    for k in range(vehicle_count):
        problem += pulp.lpSum(x[0][j][k] for j in range(1, customer_count)) == 1
        problem += pulp.lpSum(x[i][0][k] for i in range(1, customer_count)) == 1

    # Constraint n°3
    for k in range(vehicle_count):
        for j in range(customer_count):
            problem += pulp.lpSum(x[i][j][k] if i != j else 0
                                  for i in range(customer_count)) - pulp.lpSum(x[j][i][k] for i in range(customer_count)) == 0

    # Constraint n°4
    for k in range(vehicle_count):
        problem += pulp.lpSum(df.demand[j] * x[i][j][k] if i != j else 0 for i in range(customer_count) for j in range(1, customer_count)) <= vehicle_capacity

    # Constraint n°5
    subtours = []
    for i in range(2, customer_count):
        subtours += itertools.combinations(range(1, customer_count), i)

    for s in subtours:
        problem += pulp.lpSum(x[i][j][k] if i != j else 0 for i, j in itertools.permutations(s, 2) for k in range(vehicle_count)) <= len(s) - 1

    # Starting of the solving process
    if problem.solve() == 1: # If the problem was solved
        # Print the quantity of vehicles required
        print('Quantity of vehicles needed: '+str(vehicle_count))
        # Print the total distance
        print('Total distance: '+"{:.2f}".format(pulp.value(problem.objective)/1000)+"km")
        break

## Visualization : displaying with Folium
# Retrieval of every vehicles' taken segments
segments = [[] for _ in range(vehicle_count)]
for k in range(vehicle_count):
    for i in range(customer_count):
        for j in range(customer_count):
            if i != j and pulp.value(x[i][j][k]) == 1:
                segments[k].append([[df.longitude[i], df.latitude[i]],[df.longitude[j],df.latitude[j]]])
# Creation of a "coords" list, in a format that can be exploited by Folium and OpenRouteService
coords = [[] for _ in range(len(segments))]
# For each vehicle, finds the route starting segment
for k in range(len(segments)) :
    index = 0
    while segments[k][index][0] != [cities['longitude'][0], cities['latitude'][0]]:
        index += 1
    if segments[k][index][0] == [cities['longitude'][0], cities['latitude'][0]] :
        coords[k].append(segments[k][index][0])
        coords[k].append(segments[k][index][1])
        segments[k].pop(index)
# For each vehicle, finding the segments that follow each others
for k in range(len(coords)) :
    index = 0
    while len(segments[k]) > 0 :
        while segments[k][index][0] != coords[k][-1] :
            index += 1
        if segments[k][index][0] == coords[k][-1] :
            coords[k].append(segments[k][index][1])
            segments[k].pop(index)
            index = 0

print("coords = "+str(coords))

Quantity of vehicles needed: 2
Total distance: 3814.64km
coords = [[[2.4753272179034886, 48.95656958120599], [4.84139, 45.7589], [5.37639, 43.2967], [7.25, 43.7], [3.88333, 43.6], [1.43333, 43.6], [-0.566667, 44.8333], [-1.55, 47.2167], [2.4753272179034886, 48.95656958120599]], [[2.4753272179034886, 48.95656958120599], [2.34445, 48.86], [7.75, 48.5833], [3.06667, 50.6333], [2.4753272179034886, 48.95656958120599]]]


The following code uses PuLP, a linear problem modeler module that interfaces with solvers, for Python.

### Usage

As an input, it takes the available number of vehicle, their cargo capacity, cities to be delivered (their GPS coordinates (longitude and latitude) as well as their amount of demand) and the GPS coordinates to the depot.

Two ways can be used by this program to obtain the distances between GPS points in order to construct the graph:
- either the Haversine formula is used to calculate the distance "as the crow flies",
- or an OpenRouteService server can be used in order to obtain the actual distance, driving a vehicle on the road using the shortest route found.

### Performance

Using OpenRouteService will produce a result that is more accurate, but it takes more time. However, no matter the choice, the key limiting factor to this way of finding a solution to a CVRP instance is the use of a solver.

Indeed, while solving an instance that contains around 10 cities to deliver to is a matter of seconds, the time expands exponentially as more cities get input. For example, it took us 7 hours to complete an instance that contained 15 cities. Furthemore, the amount of RAM needed also expands exponentially. This makes it impossible to solve instances of the sizes expected for this project. 

## Using a metaheuristic

For this reason, our final program will not use a solver software, but a metaheuristic.
For big instances, metaheuristics are way faster than using a solver. The only downside to metaheuristics is that they don't always give the best possible solution, but they produce solutions whose quality are perfectly acceptable. 